In [57]:
import pandas as pd
import numpy as np
movies_df = pd.read_csv('ml-latest-small/ml-latest-small/movies.csv',usecols=['movieId','title'],dtype={'movieId': 'int32', 'title': 'str'})
movies_df.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [58]:
rating_df=pd.read_csv('ml-latest-small/ml-latest-small/ratings.csv',usecols=['userId', 'movieId', 'rating'],
    dtype={'userId': 'int32', 'movieId': 'int32', 'rating': 'float32'})
rating_df.head()

,userId,movieId,rating
0,1,1,4.000
1,1,3,4.000
2,1,6,4.000
3,1,47,5.000
4,1,50,5.000


In [59]:
df = pd.merge(rating_df,movies_df,on='movieId')
df.head()

,userId,movieId,rating,title
0,1,1,4.000,Toy Story (1995)
1,1,3,4.000,Grumpier Old Men (1995)
2,1,6,4.000,Heat (1995)
3,1,47,5.000,Seven (a.k.a. Se7en) (1995)
4,1,50,5.000,"Usual Suspects, The (1995)"


In [60]:
combine_movie_rating = df.dropna(axis = 0, subset = ['title'])
movie_ratingCount = (combine_movie_rating.groupby(by = ['title'])
     ['rating'].count().reset_index().
     rename(columns = {'rating': 'totalRatingCount'})
     [['title', 'totalRatingCount']]
    )
movie_ratingCount.head()

,title,totalRatingCount
0,'71 (2014),1
1,'Hellboy': The Seeds of Creation (2004),1
2,'Round Midnight (1986),2
3,'Salem's Lot (2004),1
4,'Til There Was You (1997),2


In [61]:
rating_with_totalRatingCount = combine_movie_rating.merge(movie_ratingCount, left_on = 'title', right_on = 'title', how = 'left')
rating_with_totalRatingCount.head()

,userId,movieId,rating,title,totalRatingCount
0,1,1,4.000,Toy Story (1995),215
1,1,3,4.000,Grumpier Old Men (1995),52
2,1,6,4.000,Heat (1995),102
3,1,47,5.000,Seven (a.k.a. Se7en) (1995),203
4,1,50,5.000,"Usual Suspects, The (1995)",204


In [62]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
print(movie_ratingCount['totalRatingCount'].describe())

count   9719.000
mean      10.375
std       22.406
min        1.000
25%        1.000
50%        3.000
75%        9.000
max      329.000
Name: totalRatingCount, dtype: float64


In [63]:
popularity_threshold = movie_ratingCount['totalRatingCount'].mean()+ 2*movie_ratingCount['totalRatingCount'].std()
rating_popular_movie= rating_with_totalRatingCount.query('totalRatingCount >= @popularity_threshold')
rating_popular_movie.head()
##print(popularity_threshold)

,userId,movieId,rating,title,totalRatingCount
0,1,1,4.000,Toy Story (1995),215
2,1,6,4.000,Heat (1995),102
3,1,47,5.000,Seven (a.k.a. Se7en) (1995),203
4,1,50,5.000,"Usual Suspects, The (1995)",204
7,1,110,4.000,Braveheart (1995),237


In [64]:
rating_popular_movie.shape

(37847, 5)

In [65]:
movie_features_df=rating_popular_movie.pivot_table(index='title',columns='userId',values='rating').fillna(0)
movie_features_df.head(186)

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
12 Angry Men (1957),0.000,0.000,0.000,5.000,0.000,0.000,0.000,0.000,0.000,0.000,...,5.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2001: A Space Odyssey (1968),0.000,0.000,0.000,0.000,0.000,0.000,4.000,0.000,0.000,0.000,...,0.000,0.000,5.000,0.000,0.000,5.000,0.000,3.000,0.000,4.500
28 Days Later (2002),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,3.500,0.000,5.000
300 (2007),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,3.000,...,0.000,0.000,0.000,0.000,3.000,0.000,0.000,5.000,0.000,4.000
"40-Year-Old Virgin, The (2005)",0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,4.500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
In the Line of Fire (1993),0.000,0.000,0.000,0.000,4.000,4.000,0.000,0.000,0.000,0.000,...,0.000,5.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Inception (2010),0.000,4.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,5.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,4.000
"Incredibles, The (2004)",0.000,0.000,0.000,0.000,0.000,0.000,4.000,0.000,0.000,2.500,...,5.000,0.000,0.000,0.000,0.000,4.000,0.000,4.000,0.000,5.000


In [66]:
from scipy.sparse import csr_matrix

movie_features_df_matrix = csr_matrix(movie_features_df.values)

from sklearn.neighbors import NearestNeighbors


##model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn = NearestNeighbors(metric = 'euclidean', algorithm = 'brute')
model_knn.fit(movie_features_df_matrix)
print(model_knn)

NearestNeighbors(algorithm='brute', metric='euclidean')


In [67]:
movie_features_df.shape

(383, 606)

In [68]:
query_index = 185 ##np.random.choice(movie_features_df.shape[0])
##print(query_index)
distances, indices = model_knn.kneighbors(movie_features_df.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)
print(distances)
print(indices)

[[ 0.         32.93554306 39.19821548 39.58535004 39.62638092 40.77069855]]
[[185 186 343 278 109 147]]


In [71]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for euclidean distance of {0}:\n'.format(movie_features_df.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, movie_features_df.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for euclidean distance of Indiana Jones and the Last Crusade (1989):

1: Indiana Jones and the Temple of Doom (1984), with distance of 32.935543060302734:
2: Terminator, The (1984), with distance of 39.19821548461914:
3: Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981), with distance of 39.585350036621094:
4: Die Hard (1988), with distance of 39.626380920410156:
5: Ghostbusters (a.k.a. Ghost Busters) (1984), with distance of 40.77069854736328:
